In [1]:
import pymysql
import pandas as pd
import numpy as np
import urllib.request, json
import nest_asyncio

In [2]:
nest_asyncio.apply()

In [3]:
#grab the entire table from sql
#contains recent traded option log and updated gex/vex/ddoi values
#make this take an argument in case we have more tables in the future
def get_sql_df():
    Host = 'deribit-db-instance.csduabs5w5go.us-east-1.rds.amazonaws.com'
    User = 'admin'
    Password = 'Deribit123'
    Database = 'deribit'
    con = pymysql.connect(user=User, password=Password,host=Host, database=Database, port=3306, ssl_key='../../Downloads/deribit-key-pair.pem')
    try:
        with con.cursor() as cur:
            query = 'SELECT * FROM deribit.option_table'
            cur.execute(query)
            df = pd.read_sql(query, con)
            return df
    finally:
        con.close()

In [4]:
df = get_sql_df()
df.head()

,MyUnknownColumn,trade_seq,trade_id,timestamp,tick_direction,price,mark_price,iv,instrument_name,index_price,direction,amount,gex,vex,ddoi
0,0,3,64622797,1613793025351,0,0.1625,0.245046,107.69,BTC-26MAR21-54000-C,55753.42,buy,10,10,10,0
1,0,2,64649826,1613851501155,0,0.1340,0.073299,165.31,BTC-5MAR21-57000-P,56738.42,buy,3,20,15,0
2,0,50,64649954,1613851546122,0,0.1370,0.063862,198.70,BTC-5MAR21-57000-C,56731.02,buy,3,30,16,0
3,0,57,64650074,1613851603336,0,0.0695,0.061493,134.11,BTC-5MAR21-60000-C,56796.98,buy,3,40,20,0
4,0,336,64657047,1613855054463,2,0.0560,0.151207,39.36,BTC-26FEB21-54000-C,56715.78,sell,2,30,10,0


In [5]:
df.shape

(6, 15)

In [6]:
testingdf = df.copy()

In [7]:
#working function here
def get_instrument():
    for x in df['instrument_name']:
        url = 'https://test.deribit.com/api/v2/public/get_instrument?instrument_name='
        url += x
        with urllib.request.urlopen(url) as url:
            data = json.loads(url.read().decode())
        result = data['result']
        if(result['is_active']):
            pass
        else:
            iname = result['instrument_name']
            testingdf.drop(testingdf[testingdf['instrument_name'] == iname].index, inplace = True)

In [8]:
get_instrument()

In [9]:
testingdf

,MyUnknownColumn,trade_seq,trade_id,timestamp,tick_direction,price,mark_price,iv,instrument_name,index_price,direction,amount,gex,vex,ddoi
0,0,3,64622797,1613793025351,0,0.1625,0.245046,107.69,BTC-26MAR21-54000-C,55753.42,buy,10,10,10,0
1,0,2,64649826,1613851501155,0,0.1340,0.073299,165.31,BTC-5MAR21-57000-P,56738.42,buy,3,20,15,0
2,0,50,64649954,1613851546122,0,0.1370,0.063862,198.70,BTC-5MAR21-57000-C,56731.02,buy,3,30,16,0
3,0,57,64650074,1613851603336,0,0.0695,0.061493,134.11,BTC-5MAR21-60000-C,56796.98,buy,3,40,20,0


In [10]:
testrow = testingdf.iloc[-1]

In [11]:
testrow

MyUnknownColumn                     0
trade_seq                          57
trade_id                     64650074
timestamp               1613851603336
tick_direction                      0
price                          0.0695
mark_price                  0.0614931
iv                             134.11
instrument_name    BTC-5MAR21-60000-C
index_price                     56797
direction                         buy
amount                              3
gex                                40
vex                                20
ddoi                                0
Name: 3, dtype: object

In [12]:
gex = testingdf.iloc[-1][12]

In [13]:
gex

40

In [14]:
vex = testingdf.iloc[-1][13]

In [15]:
vex

20

In [16]:
import asyncio
import websockets
import json

# To subscribe to this channel:
msg = \
    {"jsonrpc": "2.0",
     "method": "public/subscribe",
     "id": 42,
     "params": {
        "channels": ["trades.option.BTC.raw"]}
    }

async def call_api(msg):
   async with websockets.connect('wss://test.deribit.com/ws/api/v2') as websocket:
       await websocket.send(msg)
       while websocket.open:
           response = await websocket.recv()
           data = response
           data = json.loads(data)
           from pandas.io.json import json_normalize
           if('result' in data):
               pass
           else:
               data_norm = json_normalize(data['params']['data'])
               #data_norm.to_csv("./option_trade_log.csv", mode='a')
               print(data_norm)
               return data_norm
               await websocket.close()
           #print("\n\n")


newdftest = asyncio.get_event_loop().run_until_complete(call_api(json.dumps(msg)))

   trade_seq  trade_id      timestamp  tick_direction   price    mark_price  \
0         74  65963909  1614651787853               2  0.0005  3.700000e-07   

       iv     instrument_name  index_price direction  amount  
0  191.33  BTC-2MAR21-45000-P     49348.41       buy     1.0  


<ipython-input-16-9b2700c24ece>:25: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  data_norm = json_normalize(data['params']['data'])


In [17]:
newdftest

,trade_seq,trade_id,timestamp,tick_direction,price,mark_price,iv,instrument_name,index_price,direction,amount
0,74,65963909,1614651787853,2,0.0005,3.700000e-07,191.33,BTC-2MAR21-45000-P,49348.41,buy,1.0


In [19]:
appenddf = testingdf.append(newdftest)

In [ ]:
import asyncio
import websockets
import json

# To subscribe to this channel:
msg = \
    {"jsonrpc": "2.0",
     "method": "public/subscribe",
     "id": 42,
     "params": {
        "channels": ["trades.option.BTC.raw"]}
    }

async def call_api(msg):
   async with websockets.connect('wss://test.deribit.com/ws/api/v2') as websocket:
       await websocket.send(msg)
       while websocket.open:
           response = await websocket.recv()
           data = response
           data = json.loads(data)
           from pandas.io.json import json_normalize
           if('result' in data):
               pass
           else:
               data_norm = json_normalize(data['params']['data'])
               #data_norm.to_csv("./option_trade_log.csv", mode='a')
               print(data_norm)
               #return data_norm

           print("\n\n")


asyncio.get_event_loop().run_until_complete(call_api(json.dumps(msg)))

<ipython-input-20-e8e8e166fcbb>:25: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  data_norm = json_normalize(data['params']['data'])
<ipython-input-21-49f87488901e>:25: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  data_norm = json_normalize(data['params']['data'])
<ipython-input-21-49f87488901e>:25: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  data_norm = json_normalize(data['params']['data'])
<ipython-input-21-49f87488901e>:25: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  data_norm = json_normalize(data['params']['data'])
<ipython-input-21-49f87488901e>:25: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  data_norm = json_normalize(data['params']['data'])


   trade_seq  trade_id      timestamp  tick_direction  price  mark_price  \
0         36  65786384  1614562825997               2  0.006    0.007993   

      iv     instrument_name  index_price direction  amount  
0  75.16  BTC-1MAR21-46000-P     46235.44      sell     0.6  
   trade_seq  trade_id      timestamp  tick_direction  price  mark_price  \
0         36  65786384  1614562825997               2  0.006    0.007993   

      iv     instrument_name  index_price direction  amount  
0  75.16  BTC-1MAR21-46000-P     46235.44      sell     0.6  



   trade_seq  trade_id      timestamp  tick_direction  price  mark_price  \
0          3  65786385  1614562826004               3  0.003    0.013975   

      iv     instrument_name  index_price direction  amount  
0  16.81  BTC-2MAR21-46000-P     46235.44      sell     0.6  



   trade_seq  trade_id      timestamp  tick_direction  price  mark_price  \
0         86  65786386  1614562826005               2  0.008    0.024244   

      iv  

<ipython-input-21-49f87488901e>:25: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  data_norm = json_normalize(data['params']['data'])


   trade_seq  trade_id      timestamp  tick_direction   price  mark_price  \
0         33  65787851  1614563394701               0  0.0255    0.015129   

       iv     instrument_name  index_price direction  amount  
0  202.24  BTC-1MAR21-46000-C     46390.84       buy     0.1  





<ipython-input-21-49f87488901e>:25: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  data_norm = json_normalize(data['params']['data'])


...

   trade_seq  trade_id      timestamp  tick_direction  price  mark_price  \
0          1  65799828  1614568033844               1  0.007    0.007433   

      iv     instrument_name  index_price direction  amount  
0  157.4  BTC-2MAR21-51000-C     46487.97      sell     1.0  





<ipython-input-21-49f87488901e>:25: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  data_norm = json_normalize(data['params']['data'])


   trade_seq  trade_id      timestamp  tick_direction  price  mark_price  \
0         40  65802135  1614569177792               3  0.006    0.006249   

       iv     instrument_name  index_price direction  amount  
0  119.03  BTC-1MAR21-46000-P     46565.45      sell     3.0  





<ipython-input-21-49f87488901e>:25: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  data_norm = json_normalize(data['params']['data'])


   trade_seq  trade_id      timestamp  tick_direction  price  mark_price  \
0         35  65802999  1614569729948               2  0.023    0.014364   

       iv     instrument_name  index_price direction  amount  
0  202.51  BTC-1MAR21-46000-C     46449.04       buy     1.0  



